<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/a_Surname_GEDCOM_Utility_stable_v_230815_1313exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# !pip install kwargs
# !pip install pandas
# !pip install python-gedcom
# section**************************************************1
import glob
from gedcom.element.individual import IndividualElement
from gedcom.parser import Parser
import pandas as pd

def select_gedcom_file():
    gedcom_files = glob.glob('*.ged')
    if not gedcom_files:
        return ''

    print("List of GEDCOM files:")
    for i, file_name in enumerate(gedcom_files, 1):
        print(f"{i}. {file_name}")

    while True:
        try:
            selected_num = int(input("Enter the number of the GEDCOM file you want to use: "))
            if 1 <= selected_num <= len(gedcom_files):
                return gedcom_files[selected_num - 1]
            else:
                print("Invalid number. Please enter a valid number from the list.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

# Initialize the last_prime_surname variable
last_prime_surname = None

class Gedcom:
    def __init__(self, file_name):
        print("Gedcom.__init__ called with file_name:", file_name)
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

def main():
    # Select the GEDCOM file
    gedcom_file_path = select_gedcom_file()

    # Create an instance of the Gedcom class
    gedcom_instance = Gedcom(gedcom_file_path)

    # Pass the GEDCOM file path to the parse_gedcom method
    gedcom_instance.parse_gedcom(gedcom_file_path=gedcom_file_path)

if __name__ == "__main__":
    main()

    @staticmethod
    def get_standard_name(file_path):
        file_name = file_path.split('/')[-1]
        if '.' in file_name:
            file_name = file_name.rsplit('.', 1)[0]
        standard_name = file_name.replace(' ', '_').lower()
        return standard_name

class Gedcom:
    def __init__(self, file_name):
        print("Gedcom.__init__ called with file_name:", file_name)
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []

    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            gedcom_lines = f.readlines()

        current_dataset = None
        npfx_count = 0
        total_count = 0
        for line in gedcom_lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0])
            tag = parts[1]
            value = parts[2] if len(parts) > 2 else None

            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME', 'FAMC']:
                    current_key = tag
                    current_dataset.add_extractable_detail(current_key, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)

        self.filter_pool = self.gedcom_datasets

        print(f'Found {total_count} total records')

class Gedcom:

    def build_ancestral_lines(self, prime_value, records):
        # Initialize a storage area for the ancestral lines
        ancestral_lines = {}

        # Iterate over each record in the filter pool
        for record in self.filter_pool:
            # Initialize a list to store the possible branches for this record
            possible_branches = []

            # Recursively search the full available ancestral family tree
            search_ancestral_tree(record, prime_value, [], possible_branches, records)

            # Initialize a dictionary to store the scores for each branch
            branch_scores = {}

            # Initialize a variable to store the prime DNA line
            prime_dnaline = None

            # Iterate over each possible branch
            for i, branch in enumerate(possible_branches):
                # Check if the branch contains the prime value
                if prime_value in branch:
                    # The branch contains the prime value, so give it a score of 1
                    branch_scores[i] = 1

                    # Set this branch as the prime DNA line
                    prime_dnaline = branch
                else:
                    # The branch does not contain the prime value, so give it a score of 0
                    branch_scores[i] = 0

            # Store the prime DNA line and branch scores for this record in the ancestral lines storage area
            ancestral_lines[record.get_gen_person()] = {
                'prime_dnaline': prime_dnaline,
                'branch_scores': branch_scores,
            }

        # Return the ancestral lines storage area
        return ancestral_lines

def search_ancestral_tree(record, prime_value, current_branch, possible_branches, records):
    # Add the current record to the current branch
    current_branch.append(record)

    # Check if the current record contains the prime value
    if record.contains_prime_value(prime_value):
        # The current record contains the prime value, so add the current branch to the list of possible branches
        possible_branches.append(current_branch)
    else:
        # The current record does not contain the prime value, so continue searching its ancestors
        for ancestor in record.get_ancestors(records):
            search_ancestral_tree(ancestor, prime_value, current_branch.copy(), possible_branches, records)

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}

    def contains_prime_value(self, prime_value):
        # Check if this instance of GedcomDataset contains the prime value
        # Return True if it does, and False otherwise
        return prime_value in self.extractable_detail.values()

    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value

    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        first_name, last_name = name.split('/', 1)
        first_name = first_name.split(' ')[0]
        last_name = last_name.rstrip('/')
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        return self.gen_person.strip('@')

    def get_anchor_gen1(self):
        return self.anchor_gen1

    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')

    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value)
            return cm_value
        except ValueError:
            return 'error'

    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX', '')
        if '&' in npfx_value:
            sort_value = npfx_value.split('&')[1].strip()
            return sort_value
        else:
            return ''

    def get_ancestors(self, records):
        # Initialize a list to store the ancestors
        ancestors = []

        # Get the ID of the family record associated with this individual
        famc_id = self.get_extractable_FAMC()

        # Look up the family record in the records dictionary
        if famc_id in records:
            fam_record = records[famc_id]

            # Extract the IDs of the mother and father from the family record
            wife_start = fam_record.find('1 WIFE @') + 8
            wife_end = fam_record.find('@', wife_start)
            mother_id = fam_record[wife_start:wife_end]

            husb_start = fam_record.find('1 HUSB @') + 8
            husb_end = fam_record.find('@', husb_start)
            father_id = fam_record[husb_start:husb_end]

            # Look up the mother and father records in the records dictionary and add them to the ancestors list
            if mother_id in records:
                mother_record = records[mother_id]
                ancestors.append(mother_record)
            if father_id in records:
                father_record = records[father_id]
                ancestors.append(father_record)

        # Return the list of ancestors
        return ancestors


    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC', '').strip('@')

    def get_ancestors(self):
        # Initialize a list to store the ancestors
        ancestors = []

        # Get the ID of the family record associated with this individual
        famc_id = self.get_extractable_FAMC()

        # Look up the family record in the records dictionary
        if famc_id in records:
            fam_record = records[famc_id]

            # Extract the IDs of the mother and father from the family record
            wife_start = fam_record.find('1 WIFE @') + 8
            wife_end = fam_record.find('@', wife_start)
            mother_id = fam_record[wife_start:wife_end]

            husb_start = fam_record.find('1 HUSB @') + 8
            husb_end = fam_record.find('@', husb_start)
            father_id = fam_record[husb_start:husb_end]

            # Look up the mother and father records in the records dictionary and add them to the ancestors list
            if mother_id in records:
                mother_record = records[mother_id]
                ancestors.append(mother_record)
            if father_id in records:
                father_record = records[father_id]
                ancestors.append(father_record)

        # Return the list of ancestors
        return ancestors

# Function definitions
def extract_id(record):
    id_start = record.find('@') + 1
    id_end = record.find('@', id_start)
    return record[id_start:id_end]

def extract_name(record):
    name_start = record.find('1 NAME ') + 6
    name_end = record.find('\n', name_start)
    name = record[name_start:name_end]
    first_name, last_name = name.split('/', 1)
    first_name = first_name[:10] # Use slicing syntax to extract the first 10 characters of the first_name variable
    last_name = last_name[:10].rstrip('/') # Use slicing syntax to extract the first 10 characters of the last_name variable
    return last_name.replace(" ", "") + first_name.replace(" ", "")

def find_parents(individual_id, generation, records):
    if individual_id not in records:
        return
    record = records[individual_id]
    famc_start = record.find('1 FAMC @') + 8
    famc_end = record.find('@', famc_start)
    famc_id = record[famc_start:famc_end]
    if famc_id not in records:
        return

    fam_record = records[famc_id]
    wife_start = fam_record.find('1 WIFE @') + 8
    wife_end = fam_record.find('@', wife_start)
    mother_id = fam_record[wife_start:wife_end]

    husb_start = fam_record.find('1 HUSB @') + 8
    husb_end = fam_record.find('@', husb_start)
    father_id = fam_record[husb_start:husb_end]

    if mother_id and mother_id in records:
        mother_record = records[mother_id]
        mother_name = extract_name(mother_record)
    else:
        mother_name = None

    if father_id and father_id in records:
        father_record = records[father_id]
        father_name = extract_name(father_record)
    else:
        father_name = None
# section**************************************************3
    if mother_name is not None and father_name is not None:
        parent_pair = father_name + "&" + mother_name
        if parent_pair not in visited_pairs:
            visited_pairs.add(parent_pair)
            if has_both_parents(records, mother_id, father_id):
                generation_table.loc[len(generation_table)] = [generation, parent_pair]

    if mother_id:
        find_parents(mother_id, generation + 1, records)

    if father_id:
        find_parents(father_id, generation + 1, records)

def has_both_parents(records, mother_id, father_id):
    return mother_id in records and father_id in records

visited_pairs = set()


def process_individual(individual_id, gedcom_instance):
    global generation_table
    generation_table = pd.DataFrame(columns=['Generation', 'Parent Pair'])
    global visited_pairs
    visited_pairs = set()



def input_prime_surname(last_prime_surname=None):
    if last_prime_surname:
        last_name = input(f"Enter prime_surname (default: {last_prime_surname}): ")
        if not last_name:
            last_name = last_prime_surname
    else:
        last_name = input("Enter prime_surname: ")
    return last_name

# Call the function to let the user input prime_surname
prime_surname = input_prime_surname(last_prime_surname)

# Call the function to let the user select the GEDCOM file
gedcom_file_path = select_gedcom_file()
if gedcom_file_path:
    # Use the selected GEDCOM file path to create an instance of the Gedcom class
    print(Gedcom)
    gedcom_instance = Gedcom(gedcom_file_path)
    gedcom_instance.parse_gedcom()

    # Call the build_ancestral_lines method to build the ancestral lines for each filtered record
    ancestral_lines = gedcom_instance.build_ancestral_lines(prime_surname, records)

    # Initialize the list of individuals
    individuals = []

    # Iterate over the filter_pool list and add each individual's last name and ID to the individuals list
    for dataset in gedcom_instance.filter_pool:
        individual_id = dataset.get_gen_person()
        last_name = dataset.get_anchor_gen1()
        individuals.append((last_name, individual_id))

    print(f'Total records found: {len(individuals)}')

    # Read the GEDCOM file and split it into individual and family records
    with open(gedcom_file_path, 'r') as file:
        data = file.read()
    data = data.split('\n0 ')
    records = {extract_id(record): record for record in data}

    # Global variables
    generation_table = None
    visited_pairs = None

# Run the process_individual function for each individual in the individuals list and create a combined DataFrame
combined_df_rows = []
for name, individual_id in individuals:
    individual_data = process_individual(individual_id, gedcom_instance)
    cm = individual_data['cM']
    sort = individual_data['Sort']
    parent_pairs_a10 = individual_data['Parent Pairs A10']
    most_distant_ancestor = generation_table.iloc[0]['Parent Pair']  # Get the most distant ancestor from the last row of generation_table
    combined_df_rows.append([individual_id, name, sort, cm, most_distant_ancestor, parent_pairs_a10])

combined_df = pd.DataFrame(combined_df_rows, columns=['ID#', 'Name', 'Match to', 'cM', 'Most Distant Ancestor', 'Ancestral Line A10'])

# Function to create hotlinks
def create_hotlink(row):
    url_base = "https://yates.one-name.net/tng/verticalchart.php?personID="
    person_id = row['ID#']
    hotlink = f'<a href="{url_base}{person_id}&tree=tree1&parentset=0&display=vertical&generations=8" target="_blank">{person_id}</a>'
    return hotlink

# Apply the hotlink function to create the 'LUN#' column
combined_df['LUN#'] = combined_df.apply(lambda row: create_hotlink(row), axis=1)

# Change the order of the columns
combined_df = combined_df[['ID#', 'Name', 'Match to', 'cM', 'Most Distant Ancestor', 'LUN#', 'Ancestral Line A10']]

# Adjust index to start from 1 instead of 0
combined_df.index = combined_df.index + 1

# Print all records from the DataFrame


List of GEDCOM files:
1. hukefamily210109.ged
2. yates-one-name-study.ged
Enter the number of the GEDCOM file you want to use: 2
Gedcom.__init__ called with file_name: yates-one-name-study.ged


AttributeError: ignored

In [16]:
!pip install kwargs
!pip install pandas
!pip install python-gedcom